# Analyse de données avec Python
## Formatage des données
Questions
* Comment gérer les valeurs non définies ou nulles?
* Comment sauvegarder un dataframe?

Objectifs
* Créer une copie d'un dataframe.
* Transformer ou éliminer les valeurs non définies ou nulles.
* Écrire les données manipulées dans un fichier CSV.

## Charger nos données

In [ ]:
# Charger le module pandas
import pandas

# Charger les données
surveys_df = pandas.read_csv('../data/surveys.csv')

## Sélection et nettoyage des valeurs non définies

In [ ]:
# Pour chaque valeur, déterminer si non définie
surveys_df.isna()

In [ ]:
# Sélectionner les enregistrements ayant au moins une valeur NaN
masque_nan = surveys_df.isna().any(axis='columns')
surveys_df[masque_nan]

In [ ]:
# Qu'est-ce que le code suivant va retourner?
une_selection = surveys_df[surveys_df['weight'].isna()]
une_selection.groupby('species_id')['record_id'].count()

### Recréer des données manquantes

In [ ]:
def etat_par_species(df, colonne:str):
    '''
    Affiche le décompte, la moyenne et la déviation standard d'une
    colonne donnée pour chaque identifiant d'espèce de DM à NL.
    - df:      objet DataFrame
    - colonne: nom d'une colonne de df
    '''
    print(
        df.groupby('species_id')[colonne].aggregate(
            ['count', 'mean', 'std']
        ).loc['DM':'NL'],
        '\n\nDécompte total :', df[colonne].count()
    )

# Avant le nettoyage
etat_par_species(surveys_df, 'weight')

In [ ]:
# Créer une copie pour ne pas modifier l'objet original
copie_surveys = surveys_df.copy()
copie_surveys.head()

In [ ]:
# Pour une valeur moyenne stable par espèce
copie_surveys.groupby('species_id')['weight'].transform('mean')

In [ ]:
# Remplacer les valeurs manquantes par les moyennes connues
copie_surveys['weight'] = copie_surveys['weight'].fillna(
    copie_surveys.groupby('species_id')['weight'].transform('mean')
)

In [ ]:
# Avant et après le nettoyage
etat_par_species(surveys_df, 'weight')
print()  # Afficher une ligne vide
etat_par_species(copie_surveys, 'weight')

### Exercice - Nettoyage
Refaites les mêmes étapes de remplissage des valeurs
non définies, mais pour la colonne `'hindfoot_length'`.
Par contre, cette fois-ci, on veut calculer
les moyennes selon `'species_id'` et `'sex'`.

La fonction `etat_par_species_et_sex()` vous est fournie
pour afficher des statistiques avant et après le nettoyage.

(5 min.)

In [ ]:
def etat_par_species_et_sex(df, colonne:str):
    '''
    Affiche le décompte, la moyenne et la déviation standard d'une
    colonne donnée pour les 5 premières espèces et pour chaque sexe.
    - df:      objet DataFrame
    - colonne: nom d'une colonne de df
    '''
    print(
        df.groupby(
            ['species_id', 'sex']
        )[colonne].aggregate(
            ['count', 'mean', 'std']
        ).unstack().head(),
        '\n\nDécompte total :', df[colonne].count()
    )

In [ ]:
colonne = ###
etat_par_species_et_sex(surveys_df, colonne)
print()  # Afficher une ligne vide

copie_surveys[colonne] = copie_surveys[colonne].###(
    copie_surveys.groupby(
        ###
    )[colonne].###('mean')
)

etat_par_species_et_sex(copie_surveys, colonne)

### Sauvegarde après nettoyage

In [ ]:
# Ne garder que les lignes sans aucun NA
df_sans_na = copie_surveys.dropna()
df_sans_na

In [ ]:
# Sauvegarder le dataframe filtré dans un fichier CSV
df_sans_na.to_csv('surveys_sans_NA.csv', index=False)

## Résumé technique
* **Statistique descriptive par groupes selon l'index de** `df`
    * `df.groupby()[colonne].transform(fonction)`
* **Nettoyage**
    * `df.copy()`
    * `isna()`, `notna()`
    * `colonne.fillna(valeur, inplace=True)`
* **Sauvegarde**
    * `df.to_csv(nom_csv, index)`